# Fooooooooooooooocus
Colab NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Colab](https://colab.research.google.com) 部署 Fooocus 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

Colab 链接：<a href="https://colab.research.google.com/github/licyk/sd-webui-all-in-one/blob/main/fooocus_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式，并安装 [Fooocus](https://github.com/lllyasviel/Fooocus)。
2. 下载模型：下载可选列表中的模型（可选）。
3. 启动 Fooocus：启动 Fooocus，并显示访问地址。
4. 其他功能：包含自定义模型下载工具和调整内网穿透参数的工具（可选）。

## 使用
1. 在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择`GPU T4`或者其他 GPU。
2. `环境配置`单元中的选项通常不需要修改，保持默认即可。
3. 运行`环境配置`单元，默认设置下已启用`配置环境完成后立即启动 Fooocus（并挂载 Google Drive）`选项，则环境配置完成后立即启动 Fooocus，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。
4. 如果未启用`配置环境完成后立即启动 Fooocus（并挂载 Google Drive）`选项，需要运行`启动 Fooocus（图片保存在 Google Drive）`单元，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。

## 提示
1. Colab 在关机后将会清除所有数据，所以每次重新启动时必须运行`环境配置`单元才能运行`启动 Fooocus（图片保存在 Google Drive）`单元。
2. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
3. 每次启动 Colab 后必须运行`环境配置`单元，才能运行`启动 Fooocus`单元启动 Fooocus。
4. 其他功能有自定义模型下载等功能，根据自己的需求进行使用。
5. 运行`启动 Fooocus（图片保存在 Google Drive）`时将弹出 Google Drive 访问授权提示，根据提示进行授权。授权后，使用 Fooocus 生成的图片将保存在 Google Drive 的`fooocus_output`文件夹中。
6. Gradio 的内网穿透地址可在启动 Fooocus 后日志中的`Running on public URL`查看。
7. Fooocus 启动预设文件可自行编写，并上传到 Github 或者其他平台并获取下载链接后，可以替换`Fooocus 启动预设文件地址`中的链接，进行环境配置时将使用该预设文件启动 Fooocus。预设文件可自行下载下来进行参考，其他有关说明可阅读：[Customization - lllyasviel/Fooocus](https://github.com/lllyasviel/Fooocus?tab=readme-ov-file#customization)。
8. 在 Colab 的文件浏览器中，可临时上传 LoRA 模型，在`Fooocus -> models`中找到`loras`文件夹，右键可以打开上传文件菜单，上传 LoRA 模型完成后在 Fooocus 中刷新模型列表即可使用，其他模型同理。注意，Colab 在关机后将会清除数据。

In [ ]:
#@title 👇 环境配置
from typing import Union, Optional
from pathlib import Path


INIT_CONFIG = 1


def echo(*args):
    """格式化消息输出"""
    for i in args:
        print(f":: {i}")



class ARIA2:
    """基于 Aria2 的文件下载工具"""

    def __init__(self, workspace: Union[str, Path], workfolder: str) -> None:
        """基于 Aria2 的文件下载工具

        参数:
            workspace (`str`, `Path`):
                工作区路径

            workfolder (`str`):
                工作区的文件夹名称
        """
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def aria2(self, url: str, path: Union[str, Path], filename: str) -> str:
        """调用 Aria2 下载文件

        参数:
            url (`str`):
                文件的下载链接

            path (`str`, `Path`):
                将文件下载到本地的路径

            filename (`str`):
                将要下载的文件重命名

        返回值:
            `str`: 文件保存路径
        """
        import os
        file_path = os.path.join(path, filename)
        if not os.path.exists(file_path):
            echo(f"开始下载 {filename} ，路径: {file_path}")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{path}" -o "{filename}"
            if os.path.exists(file_path) and not os.path.exists(file_path + ".aria2"):
                echo(f"{filename} 下载完成")
                return file_path
            else:
                echo(f"{filename} 下载中断")
                return None
        else:
            if os.path.exists(file_path + ".aria2"):
                echo(f"开始下载 {filename} ，路径: {path}/{filename}")
                !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{path}" -o "{filename}"
                if os.path.exists(file_path) and not os.path.exists(file_path + ".aria2"):
                    echo(f"{filename} 下载完成")
                    return file_path
                else:
                    echo(f"{filename} 下载中断")
                    return None
            else:
                echo(f"{filename} 文件已存在，路径: {file_path}")
                return file_path



# GIT
class GIT:
    """基于 Git 命令的模块"""

    def __init__(self, workspace: Union[str, Path], workfolder: str) -> None:
        """基于 Git 命令的模块

        参数:
            workspace (`str`, `Path`):
                工作区路径

            workfolder (`str`):
                工作区的文件夹名称
        """
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def exists(self, addr: Optional[str]= None, path: Optional[Union[str, Path]]= None, name: Optional[str]= None) -> bool:
        """检测要克隆的项目是否存在于指定路径

        参数:
            addr (`str`, `None`):
                Git 仓库的地址

            path (`str`, `Path`, `None`):
                将 Git 仓库下载到本地的路径

            name (`str`, `None`):
                将 Git 仓库进行重命名

        返回值:
            `bool`: Git 仓库存在时则返回`True`, 否则返回`False`
        """
        import os
        if addr is not None:
            if path is None and name is None:
                path = os.path.join(os.getcwd(), addr.split("/").pop().split(".git", 1)[0])
            elif path is None and name is not None:
                path = os.path.join(os.getcwd(), name)
            elif path is not None and name is None:
                path = os.path.join(os.path.normpath(path), addr.split("/").pop().split(".git", 1)[0])
            elif path is not None and name is not None:
                path = os.path.join(os.path.normpath(path), name)

        if os.path.exists(path):
            return True
        else:
            return False


    def clone(self, addr: str, path: Optional[Union[str, Path]]= None, name: Optional[str]= None) -> str:
        """克隆 Git 仓库到本地

        参数:
            addr (`str`):
                Git 仓库的地址

            path (`str`, `Path`, `None`):
                将 Git 仓库下载到本地的路径

            name (`str`, `None`):
                将 Git 仓库进行重命名

        返回值:
            `str`: 下载的仓库地址
        """
        import os
        repo = addr.split("/").pop().split(".git", 1)[0]
        if path is None and name is None:
            path = os.getcwd()
            name = repo
        elif path is not None and name is None:
            name = repo
        elif path is None and name is not None:
            path = os.getcwd()

        repo_path = os.path.join(path, name)

        if not self.exists(addr, path, name):
            echo(f"开始下载 {repo}")
            !git clone {addr} "{repo_path}" --recurse-submodules
            if os.path.exists(repo_path):
                echo(f"{repo} 下载成功, 路径: {repo_path}")
                return repo_path
            else:
                echo(f"{repo} 下载失败")
                return None
        else:
            echo(f"{repo} 已存在, 路径: {repo_path}")
            return repo_path



class TUNNEL:
    """内网穿透工具"""
    LOCALHOST_RUN = "localhost.run"
    REMOTE_MOE = "remote.moe"

    def __init__(self, workspace: Union[str, Path], workfolder: str, port: int) -> None:
        """提供初始化环境的功能

        参数:
            workspace (`str`, `Path`):
                工作区路径

            workfolder (`str`):
                工作区的文件夹名称

            port (`int`):
                要进行端口映射的端口
        """
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder
        self.PORT = port


    def ngrok(self, ngrok_token: str) -> Union[str, None]:
        """使用 Ngrok 内网穿透

        参数:
            ngrok_token (`str`):
                Ngrok 账号 Token

        返回值:
            `str`:
                Ngrok 内网穿透生成的访问地址
        """
        from pyngrok import conf, ngrok
        conf.get_default().auth_token = ngrok_token
        conf.get_default().monitor_thread = False
        port = self.PORT
        try:
            ssh_tunnels = ngrok.get_tunnels(conf.get_default())
        except Exception as e:
            echo("启动 Ngrok 内网穿透时出现了错误", e)
            return None
        if len(ssh_tunnels) == 0:
            ssh_tunnel = ngrok.connect(port, bind_tls=True)
            return ssh_tunnel.public_url
        else:
            return ssh_tunnels[0].public_url


    def cloudflare(self) -> Union[str, None]:
        """使用 CloudFlare 内网穿透

        返回值:
            `str`:
                CloudFlare 内网穿透生成的访问地址
        """
        from pycloudflared import try_cloudflare
        port = self.PORT
        try:
            return try_cloudflare(port).tunnel
        except Exception as e:
            echo("启动 CloudFlare 内网穿透时出现了错误", e)
            return None


    def gen_key(self, path: Union[str, Path]) -> None:
        """生成 SSH 密钥

        参数:
            path (`str`, `Path`):
                生成 SSH 密钥的路径
        """
        import subprocess
        import shlex
        from pathlib import Path
        path = Path(path)
        arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
        args = shlex.split(arg_string)
        subprocess.run(args, check=True)
        path.chmod(0o600)


    def ssh_tunnel(self, host: str) -> Union[str, None]:
        """使用 SSH 进行内网穿透

        参数:
            host (`str`):
                提供 SSH 内网穿透的服务器地址

        返回值:
            (`str`, `None`):
                使用内网穿透得到的访问地址, 如果启动内网穿透失败则不返回地址
        """
        import subprocess
        import atexit
        import shlex
        import re
        import os
        from pathlib import Path
        from tempfile import TemporaryDirectory

        ssh_name = "id_rsa"
        ssh_path = Path(self.WORKSPACE) / ssh_name
        port = self.PORT

        tmp = None
        if not ssh_path.exists():
            try:
                self.gen_key(ssh_path)
            # write permission error or etc
            except subprocess.CalledProcessError:
                tmp = TemporaryDirectory()
                ssh_path = Path(tmp.name) / ssh_name
                self.gen_key(ssh_path)

        arg_string = f"ssh -R 80:127.0.0.1:{port} -o StrictHostKeyChecking=no -i {ssh_path.as_posix()} {host}"
        args = shlex.split(arg_string)

        tunnel = subprocess.Popen(
            args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)
        if tmp is not None:
            atexit.register(tmp.cleanup)

        tunnel_url = ""
        LOCALHOST_RUN = self.LOCALHOST_RUN
        lines = 27 if host == LOCALHOST_RUN else 5
        localhostrun_pattern = re.compile(r"(?P<url>https?://\S+\.lhr\.life)")
        remotemoe_pattern = re.compile(r"(?P<url>https?://\S+\.remote\.moe)")
        pattern = localhostrun_pattern if host == LOCALHOST_RUN else remotemoe_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")

            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                if lines == 27:
                    os.environ['LOCALHOST_RUN'] = tunnel_url
                    return tunnel_url
                else:
                    os.environ['REMOTE_MOE'] = tunnel_url
                    return tunnel_url
                # break
        else:
            echo(f"启动 {host} 内网穿透失败")
            return None


    def localhost_run(self) -> Union[str, None]:
        """使用 localhost.run 进行内网穿透

        返回值:
            (`str`, `None`):
                使用内网穿透得到的访问地址, 如果启动内网穿透失败则不返回地址
        """
        urls = self.ssh_tunnel(self.LOCALHOST_RUN)
        return urls


    def remote_moe(self) -> Union[str, None]:
        """使用 remote.moe 进行内网穿透
        
        返回值:
            (`str`, `None`):
                使用内网穿透得到的访问地址, 如果启动内网穿透失败则不返回地址
        """
        urls = self.ssh_tunnel(self.REMOTE_MOE)
        return urls


    def gradio(self) -> Union[str, None]:
        """使用 Gradio 进行内网穿透

        返回值:
            (`str`, `None`):
                使用内网穿透得到的访问地址, 如果启动内网穿透失败则不返回地址
        """
        import secrets
        try:
            from gradio_tunneling.main import setup_tunnel
        except Exception as e:
            echo(f"导入 Gradio Tunneling 模块时出现错误", e)
            return None

        subdomain = secrets.token_urlsafe(32)
        port = self.PORT

        try:
            tunnel_url = setup_tunnel(
                local_host="127.0.0.1",
                local_port=port,
                share_token=subdomain,
                share_server_address=None
            )
            return tunnel_url
        except Exception as e:
            echo("启动 Gradio 内网穿透失败", e)
            return None


    def start(
            self,
            ngrok: bool = False,
            ngrok_token: Optional[str] = None,
            cloudflare: bool = False,
            remote_moe: bool = False,
            localhost_run: bool = False,
            gradio: bool = False
    ) -> None:
        """启动内网穿透

        参数:
            ngrok (`bool`):
                启用 Ngrok 内网穿透

            ngrok_token (`str`, `None`):
                Ngrok 账号 Token

            cloudflare (`bool`):
                启用 CloudFlare 内网穿透

            remote_moe (`bool`):
                启用 remote.moe 内网穿透

            localhost_run (`bool`):
                使用 localhost.run 内网穿透

            gradio (`bool`):
                使用 Gradio 内网穿透
        """
        if cloudflare or (ngrok and ngrok_token) or remote_moe or localhost_run or gradio:
            echo("启动内网穿透")
        else:
            return

        cloudflare_url = self.cloudflare() if cloudflare else None
        ngrok_url = self.ngrok(ngrok_token) if ngrok and ngrok_token else None
        remote_moe_url = self.remote_moe() if remote_moe else None
        localhost_run_url = self.localhost_run() if localhost_run else None
        gradio_url = self.gradio() if gradio else None

        echo("下方为 Fooocus 访问地址")
        print("==================================================================================")
        print(f":: CloudFlare: {cloudflare_url}")
        print(f":: Ngrok: {ngrok_url}")
        print(f":: remote.moe: {remote_moe_url}")
        print(f":: localhost_run: {localhost_run_url}")
        print(f":: Gradio: {gradio_url}")
        print("==================================================================================")



class ENV:
    """提供初始化环境的功能"""


    def __init__(self, workspace: Union[str, Path], workfolder: str) -> None:
        """提供初始化环境的功能

        参数:
            workspace (`str`, `Path`):
                工作区路径

            workfolder (`str`):
                工作区的文件夹名称
        """
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def prepare_env_depend(self, use_uv: bool = False) -> None:
        """为 Notebook 的功能准备必须的环境依赖

        参数:
            use_uv (`bool`):
                是否使用 uv 代替 Pip 安装 Python 软件包
        """
        echo("安装自身组件依赖")
        !pip install uv
        pkg = "pyngrok pycloudflared gradio-tunneling"
        if use_uv:
            !uv pip install {pkg} --system --quiet -U || pip install {pkg} -U
        else:
            !pip install {pkg} -U
        !apt update
        !apt install aria2 ssh google-perftools -y


    def prepare_torch(self, torch_ver: str, xformers_ver: str, use_uv: bool = False) -> None:
        """安装 PyTorch 和 xFormers

        参数:
            torch_ver (`str`):
                PyTorch 软件包名称和版本信息, 如`torch==2.0.0 torchvision==0.15.1`

            xformers_ver (`str`):
                xFormers 软件包名称和版本信息, 如`xformers==0.0.18`

            use_uv (`bool`):
                使用 uv 代替 Pip 进行 Python 软件包安装
        """
        if use_uv:
            if torch_ver != "":
                echo("安装 PyTorch")
                !uv pip install {torch_ver} --system --quiet || pip install {torch_ver}
            if xformers_ver != "":
                echo("安装 xFormers")
                !uv pip install {xformers_ver} --no-deps --system --quiet || pip install {xformers_ver} --no-deps
        else:
            if torch_ver != "":
                echo("安装 PyTorch")
                !pip install {torch_ver}
            if xformers_ver != "":
                echo("安装 xFormers")
                !pip install {xformers_ver} --no-deps


    def install_requirements(self, path: Union[str, Path], use_uv: bool = False) -> None:
        """从文件 (requirements.txt) 中安装 Python 软件包

        参数:
            path (`str`, `Path`):
                依赖记录文件的路径

            use_uv (`bool`):
                使用 uv 代替 Pip 进行 Python 软件包安装
        """
        import os
        if os.path.exists(path):
            echo("安装依赖")
            if use_uv:
                !uv pip install -r "{path}" --system --quiet || pip install -r "{path}"
            else:
                !pip install -r "{path}"
        else:
            echo("依赖文件路径为空")


    def tcmalloc_colab(self) -> None:
        """配置 TCMalloc 内存优化"""
        echo("配置内存优化")
        import os
        aria2 = ARIA2(self.WORKSPACE, self.WORKFOLDER)
        path = self.WORKSPACE
        libtcmalloc_path = os.path.join(self.WORKSPACE, "libtcmalloc_minimal.so.4")
        aria2.aria2("https://github.com/licyk/term-sd/releases/download/archive/libtcmalloc_minimal.so.4", path, "libtcmalloc_minimal.so.4")
        os.environ["LD_PRELOAD"] = libtcmalloc_path



class MANAGER:
    """环境管理"""

    def __init__(self, workspace: Union[str, Path], workfolder: str) -> None:
        """环境管理

        参数:
            workspace (`str`, `Path`):
                工作区路径

            workfolder (`str`):
                工作区的文件夹名称
        """
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def clear_up(self) -> None:
        """清理 Notebook 的内容输出"""
        from IPython.display import clear_output
        clear_output(wait=False)


    def check_gpu(self) -> None:
        """检查环境中是否有可用的 GPU"""
        echo("检测 GPU 是否可用")
        import tensorflow as tf
        echo(f"TensorFlow 版本: {tf.__version__}")
        if tf.test.gpu_device_name():
            echo("GPU 可用")
        else:
            echo("GPU 不可用")
            raise Exception("\n没有可用的 GPU, 请在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择 GPU T4\n如果不能使用 GPU, 请尝试更换账号!")


    def config_google_drive(self) -> None:
        """配置 Google Drive"""
        echo("检查 Google Drive 是否已挂载")
        import os
        if not os.path.exists('/content/drive/MyDrive'):
            echo("挂载 Google Drive 中, 请根据提示进行操作")
            from google.colab import drive
            try:
                drive.mount('/content/drive')
                echo("Google Dirve 挂载完成")
            except Exception as e:
                raise Exception("挂载 Google Drive 时出现问题", e)
        else:
            echo("Google Drive 已挂载")

        # 检测并创建输出文件夹
        if os.path.exists('/content/drive/MyDrive'):
            if not os.path.exists('/content/drive/MyDrive/fooocus_output'):
                echo("在 Google Drive 创建 fooocus_ouput 文件夹")
                !mkdir -p /content/drive/MyDrive/fooocus_output
        else:
            raise Exception("未挂载 Google Drive, 请重新挂载后重试")



class ModelDownload:
    """模型下载器"""

    def __init__(self, urls: list) -> None:
        """模型下载器创建

        参数:
            urls (`list`):
                下载任务

        下载队列使用`ARIA2().aria2()`处理, 下载队列的格式如下:
        ```python
        urls = [
            ["url", "path", "filename"],
            ["url", "path", "filename"],
            ["url", "path", "filename"],
        ]
        ```
        """
        import threading
        import datetime
        from queue import Queue
        self.urls = urls
        self.queue = Queue()
        self.aria2 = ARIA2(None, None)
        self.datetime = datetime
        self.total_urls = len(urls)  # 记录总的 URL 数
        self.downloaded_count = 0  # 记录已下载的数量
        self.lock = threading.Lock()  # 创建锁以保护对下载计数器的访问


    def worker(self) -> None:
        """调用文件下载工具"""
        while True:
            url = self.queue.get()
            if url is None:
                break
            self.aria2.aria2(
                url=url[0],
                path=url[1],
                filename=url[2]
            )
            self.queue.task_done()
            with self.lock:  # 访问共享资源时加锁
                self.downloaded_count += 1
                self.print_progress()  # 打印进度


    def print_progress(self) -> None:
        """进度条显示"""
        progress = (self.downloaded_count / self.total_urls) * 100
        current_time = self.datetime.datetime.now()
        time_interval = current_time - self.start_time
        hours = time_interval.seconds // 3600
        minutes = (time_interval.seconds // 60) % 60
        seconds = time_interval.seconds % 60
        formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}"

        if self.downloaded_count > 0:
            speed = self.downloaded_count / time_interval.total_seconds()
        else:
            speed = 0

        remaining_urls = self.total_urls - self.downloaded_count

        if speed > 0:
            estimated_remaining_time_seconds = remaining_urls / speed
            estimated_remaining_time = self.datetime.timedelta(seconds=estimated_remaining_time_seconds)
            estimated_hours = estimated_remaining_time.seconds // 3600
            estimated_minutes = (estimated_remaining_time.seconds // 60) % 60
            estimated_seconds = estimated_remaining_time.seconds % 60
            formatted_estimated_time = f"{estimated_hours:02}:{estimated_minutes:02}:{estimated_seconds:02}"
        else:
            formatted_estimated_time = "N/A"

        echo(f"模型下载进度: {progress:.2f}% | {self.downloaded_count}/{self.total_urls} [{formatted_time}<{formatted_estimated_time}, {speed:.2f}it/s]")


    def start_threads(self, num_threads: int = 16) -> None:
        """启动多线程下载器

        参数:
            num_threads (`int`):
                下载线程, 默认为 16
        """
        import threading
        threads = []
        self.start_time = self.datetime.datetime.now()
        for _ in range(num_threads):
            thread = threading.Thread(target=self.worker)
            thread.start()
            threads.append(thread)

        for url in self.urls:
            self.queue.put(url)

        self.queue.join()

        for _ in range(num_threads):
            self.queue.put(None)

        for thread in threads:
            thread.join()



class MIRROR:
    """PyPI, HuggingFace, Github 镜像管理工具"""

    def __init__(self, workspace: Union[str, Path]) -> None:
        """设置镜像测试缓存目录和 Git 配置文件目录"""
        self.WORKSPACE = workspace


    def set_pypi_index_mirror(self, mirror: Optional[str] = None) -> None:
        """设置 PyPI Index 镜像源

        参数:
            mirror (`str`, `None`):
                PyPI 镜像源链接, 当不传入镜像源链接时则清除镜像源
        """
        import os
        if mirror:
            echo("使用 PIP_INDEX_URL, UV_INDEX_URL 环境变量设置 PyPI Index 镜像源")
            os.environ["PIP_INDEX_URL"] = mirror
            os.environ["UV_INDEX_URL"] = mirror
        else:
            echo("清除 PIP_INDEX_URL, UV_INDEX_URL 环境变量, 取消使用 PyPI Index 镜像源")
            if "PIP_INDEX_URL" in os.environ:
                del os.environ["PIP_INDEX_URL"]

            if "UV_INDEX_URL" in os.environ:
                del os.environ["UV_INDEX_URL"]


    def set_pypi_extra_index_mirror(self, mirror: Optional[str] = None) -> None:
        """设置 PyPI Extra Index 镜像源

        参数:
            mirror (`str`, `None`):
                PyPI 镜像源链接, 当不传入镜像源链接时则清除镜像源
        """
        import os
        if mirror:
            echo("使用 PIP_EXTRA_INDEX_URL, UV_EXTRA_INDEX_URL 环境变量设置 PyPI Extra Index 镜像源")
            os.environ["PIP_EXTRA_INDEX_URL"] = mirror
            os.environ["UV_EXTRA_INDEX_URL"] = mirror
        else:
            echo("清除 PIP_EXTRA_INDEX_URL, UV_EXTRA_INDEX_URL 环境变量, 取消使用 PyPI Extra Index 镜像源")
            if "PIP_EXTRA_INDEX_URL" in os.environ:
                del os.environ["PIP_EXTRA_INDEX_URL"]

            if "UV_EXTRA_INDEX_URL" in os.environ:
                del os.environ["UV_EXTRA_INDEX_URL"]


    def set_pypi_find_links_mirror(self, mirror: Optional[str] = None) -> None:
        """设置 PyPI Find Links 镜像源

        参数:
            mirror (`str`, `None`):
                PyPI 镜像源链接, 当不传入镜像源链接时则清除镜像源
        """
        import os
        if mirror:
            echo("使用 PIP_FIND_LINKS, UV_FIND_LINKS 环境变量设置 PyPI Find Links 镜像源")
            os.environ["PIP_FIND_LINKS"] = mirror
            os.environ["UV_FIND_LINKS"] = mirror
        else:
            echo("清除 PIP_FIND_LINKS, UV_FIND_LINKS 环境变量, 取消使用 PyPI Find Links 镜像源")
            if "PIP_FIND_LINKS" in os.environ:
                del os.environ["PIP_FIND_LINKS"]

            if "UV_FIND_LINKS" in os.environ:
                del os.environ["UV_FIND_LINKS"]


    def set_github_mirror(self, mirror: Optional[Union[str, list]] = None) -> None:
        """设置 Github 镜像源

        参数:
            mirror (`str`, `list`, `None`):
                Github 镜像源地址, 当传入的是 Github 镜像源地址, 则直接设置 GIT_CONFIG_GLOBAL 环境变量并直接使用该镜像源地址

                如果传入的是镜像源列表, 则自动测试可用的 Github 镜像源并设置 GIT_CONFIG_GLOBAL 环境变量

                当不传入参数时则清除 GIT_CONFIG_GLOBAL 环境变量并删除 GIT_CONFIG_GLOBAL 环境变量对应的 Git 配置文件

        使用:
        ```python
        set_github_mirror() # 不传入参数时则清除 Github 镜像源

        set_github_mirror("https://ghfast.top/https://github.com") # 只设置一个 Github 镜像源时将直接使用该 Github 镜像源

        set_github_mirror( # 传入 Github 镜像源列表时将自动测试可用的 Github 镜像源并设置
            [
                "https://ghfast.top/https://github.com",
                "https://mirror.ghproxy.com/https://github.com",
                "https://ghproxy.net/https://github.com",
                "https://gh.api.99988866.xyz/https://github.com",
                "https://gitclone.com/github.com",
                "https://gh-proxy.com/https://github.com",
                "https://ghps.cc/https://github.com",
                "https://gh.idayer.com/https://github.com",
            ]
        )
        ```
        """
        import os
        import subprocess
        if not mirror:
            echo("清除 GIT_CONFIG_GLOBAL 环境变量, 取消使用 Github 镜像源")
            if "GIT_CONFIG_GLOBAL" in os.environ:
                # 删除对应的配置文件
                if os.path.exists(os.environ["GIT_CONFIG_GLOBAL"]):
                    path = os.environ["GIT_CONFIG_GLOBAL"]
                    !rm -rf "{path}"

                del os.environ["GIT_CONFIG_GLOBAL"]

            return

        git_config_path = os.path.join(self.WORKSPACE, ".gitconfig")
        os.environ["GIT_CONFIG_GLOBAL"] = git_config_path

        if isinstance(mirror, str):
            echo("通过 GIT_CONFIG_GLOBAL 环境变量设置 Github 镜像源")
            !git config --global url."{mirror}".insteadOf "https://github.com"
        elif isinstance(mirror, list):
            mirror_test_path = os.path.join(self.WORKSPACE, "__github_mirror_test__")
            for gh in mirror:
                echo(f"测试 Github 镜像源: {gh}")
                test_repo = f"{gh}/licyk/empty"
                if os.path.exists(mirror_test_path):
                    !rm -rf "{mirror_test_path}"
                result = subprocess.run(f'git clone {test_repo} "{mirror_test_path}"')
                if os.path.exists(mirror_test_path):
                    !rm -rf "{mirror_test_path}"
                if result.returncode == 0:
                    echo("该镜像源可用")
                    !git config --global url."{gh}".insteadOf "https://github.com"
                    return
                else:
                    echo("镜像源不可用")

            echo("无可用的 Github 镜像源, 取消使用 Github 镜像源")
            if os.path.exists(git_config_path):
                !rm -rf "{git_config_path}"
            if "GIT_CONFIG_GLOBAL" in os.environ:
                del os.environ["GIT_CONFIG_GLOBAL"]
        else:
            echo(f"未知镜像源参数类型: {type(mirror)}")
            return


    def set_huggingface_mirror(self, mirror: Optional[str] = None) -> None:
        """设置 HuggingFace 镜像源

        参数:
            mirror (`str`, `None`):
                HuggingFace 镜像源链接, 当不传入镜像源链接时则清除镜像源
        """
        import os
        if mirror:
            echo("使用 HF_ENDPOINT 环境变量设置 HuggingFace 镜像源")
            os.environ["HF_ENDPOINT"] = mirror
        else:
            echo("清除 HF_ENDPOINT 环境变量, 取消使用 HuggingFace 镜像源")
            if "HF_ENDPOINT" in os.environ:
                del os.environ["HF_ENDPOINT"]


    def set_mirror(
        self,
        pypi_index_mirror: Optional[str] = None,
        pypi_extra_index_mirror: Optional[str] = None,
        pypi_find_links_mirror: Optional[str] = None,
        github_mirror: Optional[Union[str, list]] = None,
        huggingface_mirror: Optional[str] = None
    ) -> None:
        """镜像源设置

        参数:
            pypi_index_mirror (`str`, `None`):
                PyPI Index 镜像源链接

            pypi_extra_index_mirror (`str`, `None`):
                PyPI Extra Index 镜像源链接

            pypi_find_links_mirror (`str`, `None`):
                PyPI Find Links 镜像源链接

            github_mirror (`str`, `list`, `None`):
                Github 镜像源链接或者镜像源链接列表

            huggingface_mirror (`str`, `None`):
                HuggingFace 镜像源链接
        """
        echo("配置镜像源中")
        self.set_pypi_index_mirror(pypi_index_mirror)
        self.set_pypi_extra_index_mirror(pypi_extra_index_mirror)
        self.set_pypi_find_links_mirror(pypi_find_links_mirror)
        self.set_github_mirror(github_mirror)
        self.set_huggingface_mirror(huggingface_mirror)
        echo("镜像源配置完成")



class FOOOCUS(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    """Fooocus 管理工具"""

    def __init__(self, workspace: Union[str, Path], workfolder: str) -> None:
        """Fooocus 管理工具

        参数:
            workspace (`str`, `Path`):
                工作区路径

            workfolder (`str`):
                工作区的文件夹名称
        """
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder
        self.tun = TUNNEL(workspace, workfolder, 7865)
        self.mirror = MIRROR(self.WORKSPACE)


    def get_sd_model(self, url: str, filename: Optional[str] = None) -> None:
        """下载大模型

        参数:
            url (`str`):
                模型的下载链接

            filename (`str`, `None`):
                模型下载后保存的名称
        """
        import os
        path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "checkpoints")
        filename = url.split("/").pop() if filename is None else filename
        super().aria2(url, path, filename)


    def get_lora_model(self, url: str, filename: Optional[str] = None) -> None:
        """下载 LoRA 模型

        参数:
            url (`str`):
                模型的下载链接

            filename (`str`, `None`):
                模型下载后保存的名称
        """
        import os
        path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "loras")
        filename = url.split("/").pop() if filename is None else filename
        super().aria2(url, path, filename)


    def install_config(self, preset: str, path_config: str, translation: str) -> None:
        """下载 Fooocus 配置文件"""
        import os
        path = os.path.join(self.WORKSPACE, self.WORKFOLDER)
        preset_path = os.path.join(path, "presets")
        language_path = os.path.join(path, "language")
        echo("下载配置文件")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json", preset_path, "custom.json")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json", path, "config.txt")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json", language_path, "zh.json")


    def pre_download_model(self, path: Union[str, Path]) -> None:
        """根据 Fooocus 配置文件预下载模型

        参数:
            path (`str`, `Path`):
                Fooocus 路径
        """
        import os
        import json

        if os.path.exists(path):
            try:
                with open(path, "r", encoding="utf8") as file:
                    data = json.load(file)
                    echo("预下载 Fooocus 模型中")
            except Exception:
                # json 文件格式出现问题
                data = {}
        else:
            data = {}

        sd_model_list = data.get("checkpoint_downloads") if not isinstance(data.get("checkpoint_downloads"), type(None)) else {}
        lora_list = data.get("lora_downloads") if not isinstance(data.get("lora_downloads"), type(None)) else {}
        vae_list = data.get("vae_downloads") if not isinstance(data.get("vae_downloads"), type(None)) else {}
        sd_model_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "checkpoints")
        lora_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "loras")
        vae_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "vae")

        model_url = []
        for i in sd_model_list:
            model_url.append([sd_model_list.get(i), sd_model_path, i])

        for i in lora_list:
            model_url.append([lora_list.get(i), lora_path, i])

        for i in vae_list:
            model_url.append([vae_list.get(i), vae_path, i])

        model_downloader = ModelDownload(model_url)
        model_downloader.start_threads()

        echo("预下载 Fooocus 模型完成")


    def install(
        self,
        torch_ver: str,
        xformers_ver: str,
        use_uv: bool,
        pypi_index_mirror: Optional[str] = None,
        pypi_extra_index_mirror: Optional[str] = None,
        pypi_find_links_mirror: Optional[str] = None,
        github_mirror: Optional[Union[str, list]] = None,
        huggingface_mirror: Optional[str] = None,
        repo: Optional[str] = None,
        preset: Optional[str] = None,
        path_config: Optional[str] = None,
        translation: Optional[str] = None
    ) -> None:
        """安装 Fooocus
        
        参数:
            torch_ver (`str`):
                指定的 PyTorch 软件包包名, 并包括版本号

            xformers_ver (`str`):
                指定的 xFormers 软件包包名, 并包括版本号

            use_uv (`bool`):
                使用 uv 替代 Pip 进行 Python 软件包的安装
        """
        import os
        os.chdir(self.WORKSPACE)
        repo = "https://github.com/lllyasviel/Fooocus" if repo is None else repo
        preset = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json" if preset is None else preset
        path_config = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json" if path_config is None else path_config
        translation = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json" if translation is None else translation
        req_file = os.path.join(self.WORKSPACE, self.WORKFOLDER, "requirements_versions.txt")
        config_file = os.path.join(self.WORKSPACE, self.WORKFOLDER, "presets", "custom.json")
        self.check_gpu()
        self.mirror.set_mirror(
            pypi_index_mirror=pypi_index_mirror,
            pypi_extra_index_mirror=pypi_extra_index_mirror,
            pypi_find_links_mirror=pypi_find_links_mirror,
            github_mirror=github_mirror,
            huggingface_mirror=huggingface_mirror
        )
        self.prepare_env_depend(use_uv)
        self.clone(repo, self.WORKSPACE, self.WORKFOLDER)
        os.chdir(os.path.join(self.WORKSPACE, self.WORKFOLDER))
        self.prepare_torch(torch_ver, xformers_ver, use_uv)
        self.install_requirements(req_file, use_uv)
        if use_uv: # 纠正 PyTorch 版本
            self.prepare_torch(torch_ver, xformers_ver, use_uv)

        self.install_config(
            preset=preset,
            path_config=path_config,
            translation=translation
        )
        self.tcmalloc_colab()
        self.pre_download_model(config_file)


#######################################################

#@markdown ## Fooocus 核心配置选项
#@markdown - Fooocus 分支仓库地址：
FOOOCUS_REPO = "https://github.com/lllyasviel/Fooocus" #@param {type:"string"}
#@markdown - Fooocus 启动预设文件地址：
FOOOCUS_PRESET = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json" #@param {type:"string"}
#@markdown - Fooocus 路径配置地址：
FOOOCUS_PATH_CONFIG = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json" #@param {type:"string"}
#@markdown - Fooocus 翻译文件下载地址：
FOOOCUS_TRANSLATION = "https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json" #@param {type:"string"}
#@markdown ---
#@markdown ## PyTorch 组件版本选项：
#@markdown - PyTorch：
PYTORCH_VER = "torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121" #@param {type:"string"}
#@markdown - xFormers：
XFORMERS_VER = "xformers==0.0.26.post1" #@param {type:"string"}
#@markdown ---
#@markdown ## 包管理器选项：
#@markdown - 使用 uv 作为 Python 包管理器
USE_UV = True #@param {type:"boolean"}
#@markdown - PyPI 主镜像源
PIP_INDEX_MIRROR = "https://pypi.python.org/simple" #@param {type:"string"}
#@markdown - PyPI 扩展镜像源
PIP_FIND_LINKS_MIRROR = "https://download.pytorch.org/whl/cu121/torch_stable.html" #@param {type:"string"}
#@markdown ---
#@markdown ## 内网穿透选项：
#@markdown - 使用 remote.moe 内网穿透
USE_REMOTE_MOE = True #@param {type:"boolean"}
#@markdown - 使用 localhost.run 内网穿透
USE_LOCALHOST_RUN = True #@param {type:"boolean"}
#@markdown - 使用 CloudFlare 内网穿透
USE_CLOUDFLARE = True #@param {type:"boolean"}
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
USE_NGROK = True #@param {type:"boolean"}
#@markdown - 使用 Gradio 内网穿透
USE_GRADIO = True #@param {type:"boolean"}
#@markdown - Ngrok Token
NGROK_TOKEN = ""  #@param {type:"string"}
#@markdown ---
#@markdown ## 快速启动选项：
#@markdown - 配置环境完成后立即启动 Fooocus（并挂载 Google Drive）
QUICK_LAUNCH = True #@param {type:"boolean"}
#@markdown - 不重复配置环境（当重复运行环境配置时将不会再进行安装）
NO_REPEAT_CONFIGURE_ENV = True #@param {type:"boolean"}


#######################################################


import os
os.chdir("/content")
fooocus = FOOOCUS("/content","Fooocus")
try:
    _ = fooocus_has_init
except:
    fooocus_has_init = False


if NO_REPEAT_CONFIGURE_ENV:
    if not fooocus_has_init:
        fooocus.install(
            torch_ver=PYTORCH_VER,
            xformers_ver=XFORMERS_VER,
            use_uv=USE_UV,
            pypi_index_mirror=PIP_INDEX_MIRROR,
            pypi_find_links_mirror=PIP_FIND_LINKS_MIRROR,
            repo=FOOOCUS_REPO,
            preset=FOOOCUS_PRESET,
            path_config=FOOOCUS_PATH_CONFIG,
            translation=FOOOCUS_TRANSLATION,
            # pypi_extra_index_mirror=PIP_EXTRA_INDEX_MIRROR, # Colab 的环境暂不需要以下镜像源
            # github_mirror=GITHUB_MIRROR,
            # huggingface_mirror=HUGGINGFACE_MIRROR
        )
        fooocus_has_init = True
        fooocus.clear_up()
        echo("Fooocus 运行环境配置完成")
    else:
        echo("检测到不重复配置环境已启用, 并且在当前 Colab 实例中已配置 Fooocus 运行环境, 不再重复配置 Fooocus 运行环境")
        echo("如需在当前 Colab 实例中重新配置 Fooocus 运行环境, 请在快速启动选项中取消不重复配置环境功能")
else:
    fooocus.install(
        torch_ver=PYTORCH_VER,
        xformers_ver=XFORMERS_VER,
        use_uv=USE_UV,
        pypi_index_mirror=PIP_INDEX_MIRROR,
        pypi_find_links_mirror=PIP_FIND_LINKS_MIRROR,
        repo=FOOOCUS_REPO,
        preset=FOOOCUS_PRESET,
        path_config=FOOOCUS_PATH_CONFIG,
        translation=FOOOCUS_TRANSLATION,
        # pypi_extra_index_mirror=PIP_EXTRA_INDEX_MIRROR, # Colab 的环境暂不需要以下镜像源
        # github_mirror=GITHUB_MIRROR,
        # huggingface_mirror=HUGGINGFACE_MIRROR
    )
    fooocus.clear_up()
    echo("Fooocus 运行环境配置完成")

if QUICK_LAUNCH:
    echo("启动 Fooocus 中")
    os.chdir("/content/Fooocus")
    fooocus.config_google_drive()
    fooocus.tun.start(
        ngrok=USE_NGROK,
        ngrok_token=NGROK_TOKEN,
        cloudflare=USE_CLOUDFLARE,
        remote_moe=USE_REMOTE_MOE,
        localhost_run=USE_LOCALHOST_RUN,
        gradio=USE_GRADIO
    )
    if USE_GRADIO:
        !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram --share
    else:
        !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram
    fooocus.clear_up()
    echo("已关闭 Fooocus")


In [ ]:
#@title 👇 下载模型（可选）

try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

# 模型下载
echo("下载模型中")
#@markdown 选择下载的模型：
##############################


sd_xl_base = False  #@param {type:"boolean"}
if sd_xl_base:
    fooocus.get_sd_model("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors")

anima_pencil_xl = False  #@param {type:"boolean"}
if anima_pencil_xl:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323674", "anima_pencil_xl.safetensors")

bluePencilXL  = False  #@param {type:"boolean"}
if bluePencilXL:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323375", "bluePencilXL_v050.safetensors")

AnythingXL_xl = False  #@param {type:"boolean"}
if AnythingXL_xl:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/AnythingXL_xl.safetensors")

abyssorangeXLElse_v10 = False  #@param {type:"boolean"}
if abyssorangeXLElse_v10:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/abyssorangeXLElse_v10.safetensors")

CounterfeitXL  = False  #@param {type:"boolean"}
if CounterfeitXL:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors")

animeIllustDiffusion  = False  #@param {type:"boolean"}
if animeIllustDiffusion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors")

nekorayxl = False  #@param {type:"boolean"}
if nekorayxl:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors")

animagine_XL_3 = False  #@param {type:"boolean"}
if animagine_XL_3:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors")

animagine_XL_3_1 = False  #@param {type:"boolean"}
if animagine_XL_3_1:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors")

animagine_XL_4 = True  #@param {type:"boolean"}
if animagine_XL_4:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0.safetensors")

heartOfAppleXL_v20 = False  #@param {type:"boolean"}
if heartOfAppleXL_v20:
    fooocus.get_sd_model("https://civitai.com/api/download/models/337306", "heartOfAppleXL_v30.safetensors")

heartOfAppleXL_v30 = False  #@param {type:"boolean"}
if heartOfAppleXL_v30:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors")

holodayo_xl_21 = False  #@param {type:"boolean"}
if holodayo_xl_21:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors")

kivotos_xl_20 = False  #@param {type:"boolean"}
if kivotos_xl_20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors")

sd_xl_anime_V52 = False  #@param {type:"boolean"}
if sd_xl_anime_V52:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_anime_V52.safetensors")

kohakuXLGamma_rev1  = False  #@param {type:"boolean"}
if kohakuXLGamma_rev1:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-gamma/resolve/main/kohaku-xl-gamma-rev1.safetensors")

kohakuXLBeta_beta7  = False  #@param {type:"boolean"}
if kohakuXLBeta_beta7:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLBeta_beta7.safetensors")

Kohaku_XL_Delta = False  #@param {type:"boolean"}
if Kohaku_XL_Delta:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Delta/resolve/main/kohaku-xl-delta-rev1.safetensors")

kohakuXL_Epsilon = False  #@param {type:"boolean"}
if kohakuXL_Epsilon:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon/resolve/main/kohaku-xl-epsilon-rev1.safetensors")

kohakuXL_Epsilon_rev2 = False  #@param {type:"boolean"}
if kohakuXL_Epsilon_rev2:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev2/resolve/main/kohaku-xl-epsilon-rev2.safetensors")

kohakuXL_Epsilon_rev3 = False  #@param {type:"boolean"}
if kohakuXL_Epsilon_rev3:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev3/resolve/main/kohaku-xl-epsilon-rev3.safetensors")

kohakuXL_Zeta = False  #@param {type:"boolean"}
if kohakuXL_Zeta:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Zeta/resolve/main/kohaku-xl-zeta.safetensors")

starryXLV52 = False  #@param {type:"boolean"}
if starryXLV52:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors")

BArtstyleDB = False  #@param {type:"boolean"}
if BArtstyleDB:
    fooocus.get_sd_model("https://civitai.com/api/download/models/299095", "BArtstyleDB_XL.safetensors")

BArtstyleDB_3 = False  #@param {type:"boolean"}
if BArtstyleDB_3:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBlueArchiveFlatCelluloidStyle_xlv3.safetensors")

pony_v6 = False  #@param {type:"boolean"}
if pony_v6:
    fooocus.get_sd_model("https://civitai.com/api/download/models/290640", "ponyDiffusionV6XL_v6StartWithThisOne.safetensors")

pdForAnime_v20 = False  #@param {type:"boolean"}
if pdForAnime_v20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors")

tPonynai3_v51 = False  #@param {type:"boolean"}
if tPonynai3_v51:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tPonynai3_v51WeightOptimized.safetensors")

omegaPonyXLAnime_v20 = False  #@param {type:"boolean"}
if omegaPonyXLAnime_v20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors")

Illustrious_XL_v01 = False  #@param {type:"boolean"}
if Illustrious_XL_v01:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors")

Illustrious_XL_v10 = True  #@param {type:"boolean"}
if Illustrious_XL_v10:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.0.safetensors")

noobaiXL_EpsV01 = False  #@param {type:"boolean"}
if noobaiXL_EpsV01:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors")

noobaiXL_EpsV05 = False  #@param {type:"boolean"}
if noobaiXL_EpsV05:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors")

noobaiXL_EpsV075 = False  #@param {type:"boolean"}
if noobaiXL_EpsV075:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors")

noobaiXL_EpsV077 = False  #@param {type:"boolean"}
if noobaiXL_EpsV077:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors")

noobaiXL_EpsV10 = False  #@param {type:"boolean"}
if noobaiXL_EpsV10:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors")

noobaiXL_EpsV11 = True  #@param {type:"boolean"}
if noobaiXL_EpsV11:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors")

noobaiXL_VPre01 = False  #@param {type:"boolean"}
if noobaiXL_VPre01:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors")

noobaiXL_VPre05 = False  #@param {type:"boolean"}
if noobaiXL_VPre05:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors")

noobaiXL_VPre06 = False  #@param {type:"boolean"}
if noobaiXL_VPre06:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors")

noobaiXL_VPre065S = False  #@param {type:"boolean"}
if noobaiXL_VPre065S:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors")

noobaiXL_VPre075S = False  #@param {type:"boolean"}
if noobaiXL_VPre075S:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors")

noobaiXL_VPre09R = False  #@param {type:"boolean"}
if noobaiXL_VPre09R:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors")

noobaiXL_VPre10 = True  #@param {type:"boolean"}
if noobaiXL_VPre10:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors")


##############################
fooocus.clear_up()
echo("模型下载完成")

In [ ]:
#@title 👇 启动 Fooocus（图片保存在 Google Drive）
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

echo("启动 Fooocus")
import os
os.chdir("/content/Fooocus")
fooocus.config_google_drive()
fooocus.tun.start(
    ngrok=USE_NGROK,
    ngrok_token=NGROK_TOKEN,
    cloudflare=USE_CLOUDFLARE,
    remote_moe=USE_REMOTE_MOE,
    localhost_run=USE_LOCALHOST_RUN,
    gradio=USE_GRADIO
)
if USE_GRADIO:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram --share
else:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram
fooocus.clear_up()
echo("已关闭 Fooocus")

### ✨ 其他功能

In [ ]:
#@title 👇 自定义模型下载
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

#@markdown ### 选择模型种类：
#@markdown - Stable Diffusion（大模型）模型
#@markdown - LoRA模型
model_type = "LoRA" # @param ["Stable Diffusion", "LoRA"]
#@markdown ###填写模型的下载链接：
model_url = "https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/CoolFlatColor.safetensors"  #@param {type:"string"}
#@markdown ###填写模型的名称（包括后缀名）：
model_name = "CoolFlatColor.safetensors"  #@param {type:"string"}

if model_type == "Stable Diffusion":
    fooocus.get_sd_model(
        url=model_url,
        filename=model_name if model_name != "" else model_url.split("/").pop()
    )
elif model_type == "LoRA":
    fooocus.get_lora_model(
        url=model_url,
        filename=model_name if model_name != "" else model_url.split("/").pop()
    )

In [ ]:
#@title 👇 启动 Fooocus（图片保存在 Colab）
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")
echo("启动 Fooocus")
import os
os.chdir("/content/Fooocus")
fooocus.tun.start(
    ngrok=USE_NGROK,
    ngrok_token=NGROK_TOKEN,
    cloudflare=USE_CLOUDFLARE,
    remote_moe=USE_REMOTE_MOE,
    localhost_run=USE_LOCALHOST_RUN,
    gradio=USE_GRADIO
)
if USE_GRADIO:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram --share
else:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram
fooocus.clear_up()
echo("已关闭 Fooocus")

In [ ]:
#@title 👇挂载 Google Drive 并备份图片
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")
fooocus.config_google_drive()
echo("备份图片中")
!cp -r /content/drive/MyDrive/fooocus_output/* /content/drive_/MyDrive/fooocus_output
echo("备份完成")

In [ ]:
#@title 👇远程共享方式修改
#@markdown ### 选择内网穿透的方式（对应下方的勾选框）：
#@markdown - 使用 remote.moe 内网穿透
#@markdown - 使用 localhost.run 内网穿透
#@markdown - 使用 CloudFlare 内网穿透
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
#@markdown - 使用 Gradio 内网穿透
USE_REMOTE_MOE = True #@param {type:"boolean"}
USE_LOCALHOST_RUN = True #@param {type:"boolean"}
USE_CLOUDFLARE = True #@param {type:"boolean"}
USE_NGROK = False #@param {type:"boolean"}
USE_GRADIO = True #@param {type:"boolean"}
#@markdown ### Ngrok Token：
NGROK_TOKEN = ""  #@param {type:"string"}

echo("设置共享方式完成")